# Working with Bedrock in Pixeltable

Pixeltable's Bedrock integration enables you to access AWS Bedrock foundation models directly from your tables.

### Prerequisites

- Activate Bedrock in your AWS account.
- Request access to your desired models (e.g. Claude Sonnet 3.7, Amazon Nova Pro).
- Obtain a **Bedrock API Key** from the AWS console (under Bedrock > API keys), or configure standard AWS IAM credentials.

### Important notes

- Bedrock usage may incur costs based on your Bedrock plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

First you'll need to install required libraries and configure your Bedrock credentials.

In [ ]:
%pip install -qU pixeltable boto3

In [ ]:
import getpass
import os

if 'BEDROCK_API_KEY' not in os.environ:
    os.environ['BEDROCK_API_KEY'] = getpass.getpass('Enter your Bedrock API Key: ')

# Optional: set the region if your Bedrock endpoint is not in us-east-1
# os.environ['BEDROCK_REGION_NAME'] = 'us-west-2'

Now let's create a Pixeltable directory to hold the tables for our demo.

In [ ]:
import pixeltable as pxt

# Remove the `bedrock_demo` directory and its contents, if it exists
pxt.drop_dir('bedrock_demo', force=True)
pxt.create_dir('bedrock_demo')

## Messages

Create a Table: In Pixeltable, create a table with columns to represent your input data and the columns where you want to store the results from Bedrock.

In [ ]:
from pixeltable.functions import bedrock

# Create a table in Pixeltable and add a computed column that calls Bedrock

t = pxt.create_table('bedrock_demo/chat', {'input': pxt.String})

t.add_computed_column(
    output=bedrock.converse(
        model_id='amazon.nova-pro-v1:0',
        messages=[{'role': 'user', 'content': [{'text': t.input}]}],
    )
)

In [ ]:
# Parse the response into a new column
t.add_computed_column(response=t.output.output.message.content[0].text)

In [ ]:
# Start a conversation
t.insert(
    [
        {
            'input': 'What was the outcome of the 1904 US Presidential election?'
        }
    ]
)
t.select(t.input, t.response).show()

### Learn more

To learn more about advanced techniques like RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://docs.pixeltable.com/howto/use-cases/rag-operations) tutorial.

If you have any questions, don't hesitate to reach out.